# Exploring and unifying NAICS codes in the OSHA data
### Author: Jeff Minucci
### Date: 9/9/2019

In [23]:
%matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import datetime
from tabulate import tabulate
from numba import jit
%matplotlib inline

Using matplotlib backend: Qt5Agg


In [53]:
if not 'data/osha/osha_normalized.csv':
    with zipfile.ZipFile('data/osha/osha_normalized.zip','r') as zfile:
        zfile.extractall('data/osha')
osha = pd.read_csv('data/osha/osha_normalized.csv')
osha = osha[osha.columns[1:]]
osha['naics_code'] = osha['naics_code'].astype(int, errors = 'ignore')
osha['sic_code'] = osha['sic_code'].astype(int, errors = 'ignore')

### Examining the NAICS codes reported in the osha data

#### How many workplaces have naics codes associated with them?

In [22]:
wp_naics = osha[['establishment_name', 'naics_code']].groupby(['establishment_name']).max()
percent_reported = len(wp_naics[wp_naics['naics_code']>1]) / len(wp_naics) * 100
print("Percent of workplaces with a naics code reported: {0:.0f}%".format(percent_reported))

Percent of workpalces with a naics code reported: 46%


In [56]:
osha['year'] = pd.DatetimeIndex(osha['date_sampled']).year

In [49]:
wp_naics_yr = osha[['establishment_name', 'naics_code','year']].groupby(['establishment_name', 'year']).max().reset_index()
print("Percent of workplaces with naics code reported by year:")
for yr in range(wp_naics_yr['year'].min(), wp_naics_yr['year'].max()+1):
    percent_reported = len(wp_naics_yr[(wp_naics_yr['year'] == yr) & (wp_naics_yr['naics_code']>1)])\
    / len(wp_naics_yr[wp_naics_yr['year'] == yr]) * 100
    print("{0}: {1:.0f}%".format(yr,percent_reported))

Percent of workplaces with naics code reported by year:
1984: 0%
1985: 0%
1986: 0%
1987: 0%
1988: 0%
1989: 0%
1990: 0%
1991: 1%
1992: 0%
1993: 0%
1994: 0%
1995: 0%
1996: 0%
1997: 1%
1998: 0%
1999: 1%
2000: 1%
2001: 2%
2002: 22%
2003: 98%
2004: 100%
2005: 100%
2006: 100%
2007: 100%
2008: 100%
2009: 100%
2010: 100%
2011: 100%
2012: 100%
2013: 100%
2014: 100%
2015: 100%
2016: 90%
2017: 100%
2018: 100%


#### How many workplaces have the older sic codes?

In [54]:
wp_sic = osha[['establishment_name', 'sic_code', 'naics_code']].groupby(['establishment_name']).max()
percent_reported = len(wp_sic[wp_sic['sic_code']>1]) / len(wp_sic) * 100
print("Percent of workplaces with an sic code reported: {0:.0f}%".format(percent_reported))

Percent of workplaces with an sic code reported: 82%


In [57]:
wp_sic_yr = osha[['establishment_name', 'naics_code', 'sic_code','year']].groupby(['establishment_name', 'year']).max().reset_index()
print("Percent of workplaces with sic code reported by year:")
for yr in range(wp_sic_yr['year'].min(), wp_sic_yr['year'].max()+1):
    percent_reported = len(wp_sic_yr[(wp_sic_yr['year'] == yr) & (wp_sic_yr['sic_code']>1)])\
    / len(wp_sic_yr[wp_sic_yr['year'] == yr]) * 100
    print("{0}: {1:.0f}%".format(yr,percent_reported))

Percent of workplaces with sic code reported by year:
1984: 100%
1985: 100%
1986: 100%
1987: 100%
1988: 100%
1989: 100%
1990: 100%
1991: 100%
1992: 100%
1993: 100%
1994: 100%
1995: 100%
1996: 100%
1997: 100%
1998: 100%
1999: 100%
2000: 100%
2001: 100%
2002: 100%
2003: 100%
2004: 100%
2005: 100%
2006: 100%
2007: 100%
2008: 100%
2009: 100%
2010: 100%
2011: 82%
2012: 25%
2013: 6%
2014: 4%
2015: 1%
2016: 10%
2017: 0%
2018: 0%


#### What format are the naics codes in? 
Options: 2002, 2007, 2012, 2017 <br>
2007 was the last major version, there are only a handful of changes in 2012 and 2017

In [59]:
wp_naics_yr[wp_naics_yr['year']==2003].head()

,establishment_name,year,naics_code
11,"104 WEST 40TH STREET PARTNERS, LLC",2003,561210.0
13,"110 GULF ASSOCIATES,",2003,531120.0
30,"18TH CENTURY HARDWARE COMPANY, INC.",2003,331528.0
41,21ST CENTURY PREPARATORY SCHOOL,2003,611110.0
57,3-D'S BRIDGE PAINTING CORP.,2003,237310.0


2003 data appears to be in 2007 or 2002 format. 331528 no longer exists in naics 2012 or 2017

In [79]:
wp_naics_yr[wp_naics_yr['naics_code']==211130].head()

,establishment_name,year,naics_code


No data appears to be in 2017 naics format

#### Convert sic codes to naics codes when naics codes aren't available

In [82]:
sic_naics_2002 = pd.read_csv('data/naics/1987_SIC_to_2002_NAICS.csv')